
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 3 -- LP Duality and the Primal-Dual Algorithm** <br />
$\qquad$ $\qquad$$\qquad$                   **During grading time, direct queries regarding Q-1,2 to David & Q-3 to Marc** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 03/03/2025** <br />
$\qquad$ $\qquad$$\qquad$                   Josef Jakobson, 0208282079, josefjak@chalmers.se <br />
$\qquad$ $\qquad$$\qquad$                   Zoe Opdendries, 0208100065, zoe@zaloz.se <br />



---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook and equations, wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:

\begin{alignat*}{2}
\max \ &4x_1-2x_2+5x_3+6x_4+7x_5\\
\\
\textrm{s.t} \quad
&2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 &&\leq 6\\
&2x_1 + \ \ {}x_2 - 2x_3 - \ \ x_4 - 3x_5 &&\geq -1\\
&5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 &&= 5\\
&2x_1 - 2x_2 + 5x_3 + 3x_4 + \ \ x_5 &&\leq 4\\
&\hspace{5.3cm} \vec x &&\geq \vec 0
\end{alignat*}

* (4 points) Write the LP dual of this problem.
* (3 points) Use CVXPY to compute the primal and dual optimum solutions and compare their values.
* (3 points) Check the complementary slackness conditions.

We use the Dualization recipe from the book to produce the dual:
\begin{alignat*}{2}
\min \ &6y_1 - \ \ {}y_2 + 5y_3 + 4y_4\\
\\
\textrm{s.t} \quad
&2y_1 + 2y_2 + 5y_3 + 2y_4 && \geq 4\\
&2y_1 + \ \ {}y_2 - 2y_3 - 2y_4 && \geq -2\\
&-4y_1 - 2y_2 + 4y_3 + 5y_4 && \geq 5\\
&4y_1 - \ \ {}y_2 + 4y_3 + 3y_4 && \geq 6\\
&8y_1 - 3y_2 + 2y_3 + \ \ {}y_4 && \geq 7\\
&y_1 && \geq 0\\
&y_2 && \leq 0\\
&y_3 && \in \mathbb{R}\\
&y_4 && \geq 0\\
\end{alignat*}


In [ ]:
import cvxpy as cp
import numpy as np

x = cp.Variable((5, 1), nonneg=True)
c_primal = np.array([4, -2, 5, 6, 7])
const_primal = [
    2*x[0] + 2*x[1] - 4*x[2] + 4*x[3] + 8*x[4] <= 6,
    2*x[0] + x[1] - 2*x[2] - x[3] - 3*x[4] >= -1,
    5*x[0] - 2*x[1] + 4*x[2] + 4*x[3] + 2*x[4] == 5,
    2*x[0] - 2*x[1] + 5*x[2] + 3*x[3] + x[4] <= 4,
]

objective_primal = cp.Maximize(c_primal @ x)

problem_primal = cp.Problem(objective_primal, const_primal)
solution_primal = problem_primal.solve()


y = cp.Variable((4, 1))
c_dual = np.array([6, -1, 5, 4])

const_dual = [
    2*y[0] + 2*y[1] + 5*y[2] + 2*y[3] >= 4,
    2*y[0] + y[1] - 2*y[2] - 2*y[3] >= -2,
    (-4)*y[0] - 2*y[1] + 4*y[2] + 5*y[3] >= 5,
    4*y[0] - y[1] + 4*y[2] + 3*y[3] >= 6,
    8*y[0] - 3*y[1] + 2*y[2] + y[3] >= 7,
    y[0]>=0, y[1] <= 0, y[3] >= 0
]

objective_dual = cp.Minimize(c_dual @ y)

problem_dual = cp.Problem(objective_dual, const_dual)
solution_dual = problem_dual.solve()


print(f"Solution for primal: {solution_primal}, Status: {problem_primal.status}")
print(f"Solution for dual: {solution_dual}, Status: {problem_dual.status}")        


Solution for primal: 9.220338951278364, Status: optimal
Solution for dual: 9.220338968917472, Status: optimal


The two solutions are more or less the same, with the difference likely being the result of the CVXPY library itself since by the Strong Duality Theorem they should have the exact same optimal solution. The code below will check the complementary slackness conditions for both LP's and print out the results of the constraints. They are not exact, which again is likely due to bugs or inconsistencies with the library, but close enough that we say that the conditions hold.

In [4]:
const = np.array([[2,2,-4, 4,8], [2,1,-2,-1,-3], [5,-2,4,4,2], [2,-2,5,3,1]]) # Constraints as matrix to make following implementation easier
const_dual_M = const.transpose()

# Check Complementary Slackness for Primal
for i in range(len(x.value)):
    if x.value[i] == 0:
        continue
    else:
        print(f"{const_dual_M[i].dot(y.value)[0]} should be equal to {c_primal[i]}")

# Check Complementary Slackness for Dual
for i in range(len(y.value)):
    if y.value[i] == 0:
        continue
    else:
        print(f"{const[i].dot(x.value)[0]} should be equal to {c_dual[i]}")


4.00000000445052 should be equal to 4
-1.9999999972511593 should be equal to -2
4.999999988019863 should be equal to 5
6.830508465549676 should be equal to 6
6.99999999064991 should be equal to 7
6.000000004771811 should be equal to 6
-0.999999920172545 should be equal to -1
4.9999999999999485 should be equal to 5
3.999999996461829 should be equal to 4


# Problem 2.

Consdier the LP problem:
\begin{alignat*}{2}
\max \ &6x_1 - 5x_3\\
\\
\textrm{s.t} \quad
&6x_1 - 3x_2 + x_3 &&= 2\\
&3x_1 + 4x_2 + x_3 &&\leq 5\\
&\ \ x_1 - 7x_2 &&\leq 5\\
&\hspace{2.45cm} x_1 &&\geq 0\\
&\hspace{2.45cm} x_2 &&\leq 0\\
&\hspace{2.45cm} x_3 &&\text{ unrestricted}
\end{alignat*}

* (3 points) Write the LP dual of this problem.
* (4 points) Consider the feasible solution $\vec x = (0,0,2)$ to the primal. Check if this solution is optimal by using the complementary slackness conditions to write down the corresponding dual solution.
* (3 points) Use complementary slackness to check if the primal feasible solution $\vec x = (1,0,-4)$ is optimal.

\begin{alignat*}{2}
\min \ &2y_1 + 5y_2 + 5y_3\\
\textrm{s.t} \quad
&6y_1 + 3y_2 + \ \ y_3 && \geq 6\\
&-3y_1 + 4y_2 - 7y_3 &&\leq 0\\
&\ \ y_1 + \ \ y_2 &&= -5\\
&\hspace{2.45cm} y_1 &&\in \mathbb{R}\\
&\hspace{2.45cm} y_2, y_3 &&\geq 0\\
\end{alignat*}

The solution $x^* = (0,0,2)$ being optimal for the primal means that there must exist solution $y^* = (y_1,y_2,y_3)$ such that complementary slackness holds. Since the first two variables $x_1 = 0$ and $x_2 = 0$ we only need to check that constraint $y_1 + y_2=-5$ holds. We also have that constraints 2 and 3 in the Primal are satisfied by equality so we have the additional formulas $y_2 = 0, y_3 = 0$. Thus we have the solution $y^* = (-5, 0, 0)$. This solution is not feasible since it violates constraint 1 in the dual LP: $6*-5 \not \geq 6$ thus this $x^*$ is not optimal.


To check if the solution $x^* = (1,0,-4)$ is optimal we must find solution $y^* = (y_1,y_2,y_3)$ such that CS holds. Using similar reasonig as above we get the following equation system:


$\left\{ \begin{array}{rcl}
  6y_1 + 3y_2 + y_3 = 6 \\ 
  y_1 + y_2 =5 \\
  y_2 = y_3 = 0
 \end{array}\right.$
 
 which is simplified to:
 $\left\{ \begin{array}{rcl}
  y_1= 1 \\ 
  y_1 =5 \\
 \end{array}\right.$
Which is not solvable, thus no such $y^*$ exists and so this $x^*$ is not optimal

# Problem 3.

Consider the primal-dual algorithm for vertex cover discussed in class.
* (4 points) Run the algorithm by hand on the graph in the figure below (from your previous homework). Show the values of the primal and dual variables at each iteration.
* (6 points) Implement the primal-dual algorithm as a python script to compute (approximate) vertex covers and run it for the random graph $G(100,0.1)$ from the previous homework.

<img src="https://tinyurl.com/tsnuz2c" alt="Drawing" style="width: 180px;"/>

If the image does not load, try the direct link: https://tinyurl.com/tsnuz2c

So the primal is defined as 

\begin{alignat*}{2}
\min \ &x_1 + 3x_2 + 4x_3 + 2x_4 + 2x_5 + 4x_6\\
\\
\textrm{s.t} \quad
&x_1 + x_2 &&\leq 1\\
&x_2 + x_3 &&\leq 1\\
&x_2 + x_5 &&\leq 1\\
&x_3 + x_5 &&\leq 1 \\
&x_3 + x_4 &&\leq 1 \\
&x_4 + x_5 &&\leq 1 \\
&x_5 + x_6 &&\leq 1 \\
&\hspace{2.45cm} x_1, x_2, x_3, x_4, x_5, x_6 &&\geq 0\\
\end{alignat*}

The Dual is defined as:

\begin{alignat*}{2}
\max \ & \sum_{i=1}^7 y_i \\
\\
\textrm{s.t} \quad
&y_1 &&\geq 1\\
&y_1 + y_2 + y_3 &&\geq 3\\
&y_2 + y_4 + y_5 &&\leq 4\\
&y_5 + y_6 &&\leq 2 \\
&y_3 + y_4 + y_6 + y_7 &&\leq 2 \\
&y_7 &&\leq 4 \\
&\hspace{2.45cm} y_i &&\geq 0\\
\end{alignat*}

We set the primal and dual variables to zero, x = (0,0,0,0,0,0), and y = (0,0,0,0,0,0,0):

We increase all dual variables $y_e$ until a vertex wieght constriant becomes tight. This happends when $y_e = 1/2$ for  $y_3 + y_4 + y_6 + y_7 \leq 2$, which results in x = (0,0,0,0,1,0), y = (1/2, 1/2, 1/2, 1/2, 1/2, 1/2, 1/2) and freezing $y_3, y_4, y_6, y_7$

The next step is to increase the remaining edges until they become tight again, this happends at $y_e = 1$ with the constraint $y_1 \leq 1$, resulting in x = (1,0,0,0,1,0) and y = (1, 1, 1/2, 1/2, 1, 1/2, 1/2), freezing $y_1$.

The next tightened constraint is when $y_e = 3/2$, with both $y_5 + y_6 \leq 2$ and $y_1 + y_2 + y_3 \leq 3$, resulting in x = (1,1,0,1,1,0) and y = (1, 3/2, 1/2, 1/2, 3/2, 1/2, 1/2). We pick poth since they both froze at the same time.

Since the dual constraint is tight the final awnser is: x = (1,1,0,1,1,0)


In [ ]:
import numpy as np
graph = np.loadtxt("graph.txt", dtype=int)
nr_of_edges = int(np.sum(graph) / 2) # The assignment says approx 1000 edges but should be half since graph is undirected

# Create solution vectors for vertex and edge variables
vertex_vec = np.zeros((graph.shape[0], 1))
edge_vec = np.zeros((nr_of_edges, 1))



# Creates dictionary containing the maximum value allowed for each edge incident on a vertex
node_to_max_edge_value = {}
for i in range(graph.shape[0]):
    node_to_max_edge_value[i] = 1/np.sum(graph[i])

# Returns the node with the smallest value for their edges.
# This is done since we always want to start with nodes with the largest amount of incident edges (since we have uniform weights)
# This is a greedy way to ensure that the "best" vertices are prioritised
def get_lowest_vertex():
    smallest = min(node_to_max_edge_value.values())
    for vertex, val in node_to_max_edge_value.items():
        if val == smallest:
            return vertex, val

# Convert vertex-vertex matrix to vertex-edge incidence matrix
node_edge_incidence = np.zeros((nr_of_edges, graph.shape[0]))
edge_index = 0
for i in range(graph.shape[0]):
    for j in range(i+1, graph.shape[0]):
        if graph[i, j] == 1:
            node_edge_incidence[edge_index, i] = 1
            node_edge_incidence[edge_index, j] = 1
            edge_index += 1

# Returns incident edges to a vertex
def get_incident_edges(vertex):
    return np.where(node_edge_incidence[:, vertex] == 1)[0]


# Returns neighbour vertex given source and edge
def get_incident_node(node, edge):
    incident = np.where(node_edge_incidence[edge, :] == 1)[0]
    if incident[0] == node:
        return incident[1]
    else:
        return incident[0]

frozen = np.array([])
# Runs until all edges have been frozen
while len(frozen) != nr_of_edges:
    vertex, val = get_lowest_vertex()
    incident_edges = get_incident_edges(vertex)

    # Set-Difference operation to remove frozen edges from pool
    free_edges = np.setdiff1d(get_incident_edges(vertex), frozen) 

    # If there are no free edges, all available edges from this vertex are already included in the solution and so 
    # it is reduntand to check it
    if len(free_edges) == 0:
        node_to_max_edge_value.pop(vertex)   
        continue

    for edge in free_edges:
        # Set corresponding edge in solution vector to maximum val
        edge_vec[edge] = val

        # Computes total of incident edges to the vertex to check if we have reached the total maximum of 1
        # If total is 1, freeze all incident edges, set the corresponding vertex solution variable to 1 and remove it from the dictionary.
        current_sum = np.matmul(node_edge_incidence.T, edge_vec)[vertex][0]
        if round(current_sum, 8) == 1: 
            vertex_vec[vertex] = 1
            frozen = np.append(frozen, free_edges)
            node_to_max_edge_value.pop(vertex)

            # For the current vertex, update the values of its neighbouring vertices to reallocate the maximum for their 
            # remaining unfrozen edges.
            for edge in free_edges:
                node = get_incident_node(vertex, edge)
                edges_left = get_incident_edges(node)
                diff = len(np.setdiff1d(edges_left, frozen))
                node_to_max_edge_value[node] = (1 - sum([edge_vec[edge] for edge in edges_left if edge in frozen]))/diff

            break

print(np.sum(vertex_vec))
print(np.sum(edge_vec))

# Confirm that solution is indeed a VC
vc = True
for edge in node_edge_incidence:
    nodes = np.where(edge == 1)
    if not (vertex_vec[nodes[0][0]] == 1 or vertex_vec[nodes[0][1]] == 1):
        vc = False
        break
print(f"The solution produces valid VC: {vc}")
print(len(frozen) == nr_of_edges)




True
76.0
46.98411882886201
The solution produces valid VC: True


C:\Users\jjako\AppData\Local\Temp\ipykernel_19372\3073455721.py:81: RuntimeWarning: divide by zero encountered in divide
  node_to_max_edge_value[node] = (1 - sum([edge_vec[edge] for edge in edges_left if edge in frozen]))/diff
